In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [15]:
def preprocess_data(data: pd.DataFrame, start_year, t_limit, t_col="T"):
    """
    Remove unneccesary rows due to missing values, get relavent data for trial
    """
    y_col = 'y_' + str(start_year + 1)
    data = data[(data['T_' + str(start_year)] > 0) & (data['T_' + str(start_year + 1)] > 0) & (data[y_col] > 0)]
    
    data = data.applymap(lambda x: -1 if x <= 0 else x)
    data = data[(data[f"{t_col}_{start_year}"] < t_limit)]
    data[f"{t_col}_{start_year + 1}"] = data[f"{t_col}_{start_year + 1}"].apply(lambda x: int(x >= t_limit))
  
    # in this part i'm removing unneccecary columns for analysis and converting categorial columns to str type
    columns_to_remove_1 = [c for c in data.columns if c.startswith('y_') and not c.endswith(str(start_year + 1))]
    columns_to_remove_2 = [c for c in data.columns if c.startswith('T_') and not c.endswith(str(start_year + 1))]
    columns_to_remove_3 = ['PUBID']
    columns_to_remove = columns_to_remove_1 + columns_to_remove_2 + columns_to_remove_3
    columns_to_use = [c for c in data.columns if c not in columns_to_remove]
    new_data = data[columns_to_use]
    keep_as_is_columns_starters = ['income_', 'CVC_HOURS_WK_YR_ALL_', 'y_']
    for c in columns_to_use:
      cont_flag = False
      for starter in keep_as_is_columns_starters:
        if c.startswith(starter):
          cont_flag = True
      if not cont_flag:
        new_data[c] = new_data[c].apply(str)

    return new_data

In [4]:
def transform_data(data: pd.DataFrame, start_year, t_limit):
    """
    Normalize values, calculate propensity, ...
    :data: complete dataframe
    :t_col: column of treatment
    :y_col: columns of result
    :t_limit: number at which we put the bound for having recieved treatment or not
    """
    cols = [c for c in data.columns if not c.startswith('T_') and not c.startswith('y_')]
    X = data[cols]
    X = pd.get_dummies(X)
    scaler = MinMaxScaler()
    scaler.fit(X)
    X_normalized = scaler.transform(X)

    t_col = 'T_' + str(start_year + 1)
    t = data[t_col]

    clf = LogisticRegression(random_state=0).fit(X_normalized, t)
    prob = clf.predict_proba(X_normalized)[: , 1]
    data['propensity'] = prob

    data_treated = data[data[t_col] == '1']
    data_control = data[data[t_col] == '0']
    return data_control, data_treated

In [5]:
def calc_IPW_ATT(data_0, data_1, start_year):
    year = start_year + 1
    IPW_substructed_upper = sum(data_0['y_' + str(year)] * (data_0['propensity'] / (1 - data_0['propensity'])))
    IPW_substructed_lower = sum(data_0['propensity'] / (1 - data_0['propensity']))
    IPW_ATT = data_1['y_' + str(year)].mean() - (IPW_substructed_upper / IPW_substructed_lower)
    return IPW_ATT

def calc_IPW_ATE(data_0, data_1, start_year):
  year = start_year + 1
  n = len(data_0) + len(data_1)
  IPW_ATE = (1 / n) * (sum(data_1['y_' + str(year)] / data_1['propensity']) 
                     - sum(data_0['y_' + str(year)] / (1 - data_0['propensity'])))
  return IPW_ATE

In [6]:
from sklearn.linear_model import LinearRegression
def calc_ATT_Slearner(data, start_year):
  year = start_year + 1
  y_col = 'y_' + str(year)
  t_col = 'T_' + str(start_year + 1)
  cols = [c for c in data.columns if c not in {'propensity', y_col}]
  x = data[cols]
  x = pd.get_dummies(x)
  scaler = MinMaxScaler()
  scaler.fit(x)
  x_normalized = scaler.transform(x)
  y = data[y_col]
  reg = LinearRegression().fit(x_normalized, y)

  x_treated = x[x[t_col + '_' + str(1)] == 1] 
  x_treated[t_col + '_' + str(1)] = x_treated[t_col + '_' + str(1)].apply(lambda x: 0)
  x_treated[t_col + '_' + str(0)] = x_treated[t_col + '_' + str(0)].apply(lambda x: 1)
  x_treated_normalized = scaler.transform(x_treated)
  y_pred = reg.predict(x_treated_normalized)

  data_1 = data[data[t_col] == '1']
  ATT = (1 / len(data_1)) * sum(data_1[y_col] - y_pred)
  return ATT

def calc_ATE_Slearner(data, start_year):
  year = start_year + 1
  y_col = 'y_' + str(year)
  t_col = 'T_' + str(start_year + 1)
  cols = [c for c in data.columns if c not in {'propensity', y_col}]
  x = data[cols]
  x = pd.get_dummies(x)
  scaler = MinMaxScaler()
  scaler.fit(x)
  x_normalized = scaler.transform(x)
  y = data[y_col]
  reg = LinearRegression().fit(x_normalized, y)

  x_treated = pd.get_dummies(x)
  x_control = pd.get_dummies(x)

  x_treated[t_col + '_' + str(1)] = x_treated[t_col + '_' + str(1)].apply(lambda x: 1)
  x_treated[t_col + '_' + str(0)] = x_treated[t_col + '_' + str(0)].apply(lambda x: 0)

  x_control[t_col + '_' + str(1)] = x_control[t_col + '_' + str(1)].apply(lambda x: 0)
  x_control[t_col + '_' + str(0)] = x_control[t_col + '_' + str(0)].apply(lambda x: 1)

  scaler.fit(x_treated)
  scaler.fit(x_control)
  
  y_pred_treated = reg.predict(x_treated)
  y_pred_control = reg.predict(x_control)

  ATE = (1 / len(data)) * sum(y_pred_treated - y_pred_control)
  return ATE
    

In [17]:
def calc_ATT_Tlearner(data, start_year):
  y_col = 'y_' + str(start_year + 1)
  t_col = 'T_' + str(start_year + 1)
  data_0 = data[data[t_col] == '0']
  cols = [c for c in data_0.columns if c not in {'propensity', y_col, t_col}]
  
  x_0 = data_0[cols]
  for c in cols:
    x_0[c] = x_0[c].astype(int)


  scaler = MinMaxScaler()
  scaler.fit(x_0)
  x_0_normalized = scaler.transform(x_0)

  y = data_0[y_col]

  reg = LinearRegression().fit(x_0_normalized, y)

  data_1 = data[data[t_col] == '1']
  x_1 = data_1[cols]
  for c in cols:
    x_1[c] = x_1[c].astype(int)

  x_1_normalized = scaler.transform(x_1)
  y_pred = reg.predict(x_1_normalized)
  y_true = data_1[y_col]

  ATT = (1 / len(y_true)) * sum(y_true - y_pred)
  return ATT

def calc_ATE_Tlearner(data, start_year):
  y_col = 'y_' + str(start_year + 1)
  t_col = 'T_' + str(start_year + 1)
  data_0 = data[data[t_col] == '0']
  data_1 = data[data[t_col] == '1']
  cols = [c for c in data_0.columns if c not in {'propensity', y_col, t_col}]

  x = data[cols]
  x_0 = data_0[cols]
  x_1 = data_1[cols]

  for c in cols:
    x[c] = x[c].astype(int)
    x_0[c] = x_0[c].astype(int)
    x_1[c] = x_1[c].astype(int)
  
  scaler = MinMaxScaler()
  scaler.fit(x)
  x_normalized = scaler.transform(x)

  scaler_0 = MinMaxScaler()
  scaler_0.fit(x_0)
  x_0_normalized = scaler_0.transform(x_0)

  scaler_1 = MinMaxScaler()
  scaler_1.fit(x_1)
  x_1_normalized = scaler_1.transform(x_1)

  y_0 = data_0[y_col]
  reg_0 = LinearRegression().fit(x_0_normalized, y_0)

  y_1 = data_1[y_col]
  reg_1 = LinearRegression().fit(x_1_normalized, y_1)

  y_1_pred = reg_1.predict(x_normalized)
  y_0_pred = reg_0.predict(x_normalized)

  ATE = (1 / len(data)) * sum(y_1_pred - y_0_pred)
  return ATE

In [9]:
from sklearn.neighbors import NearestNeighbors
import numpy as np


def calc_ATT_matching(data, start_year, num_neighboors):
  y_col = 'y_' + str(start_year + 1)
  t_col = 'T_' + str(start_year + 1)
  data_0 = data[data[t_col] == '0']
  data_1 = data[data[t_col] == '1']

  cols = cols = [c for c in data_0.columns if c not in {'propensity', y_col, t_col}]

  x_0 = data_0[cols]
  x_1 = data_1[cols]

  for c in cols:
    x_0[c] = x_0[c].astype(int)
    x_1[c] = x_1[c].astype(int)

  scaler_0 = MinMaxScaler()
  scaler_0.fit(x_0)
  x_0_normalized = scaler_0.transform(x_0)

  scaler_1 = MinMaxScaler()
  scaler_1.fit(x_1)
  x_1_normalized = scaler_1.transform(x_1)

  y_1 = data_1[y_col].to_numpy()
  y_0 = data_0[y_col].to_numpy()

    
  nbrs = NearestNeighbors(n_neighbors=num_neighboors, algorithm='ball_tree').fit(x_0_normalized)
  _, indices = nbrs.kneighbors(x_1_normalized)

  ITT_list = []

  for i, indices in enumerate(indices):
    closest_y_0_avg = np.mean(y_0[indices])
    ITT = y_1[i] - closest_y_0_avg
    ITT_list.append(ITT)

  ATT = np.mean(ITT_list)
  return ATT

def calc_ATE_matching(data, start_year, num_neighboors):
  y_col = 'y_' + str(start_year + 1)
  t_col = 'T_' + str(start_year + 1)
  data_0 = data[data[t_col] == '0']
  data_1 = data[data[t_col] == '1']

  cols = cols = [c for c in data_0.columns if c not in {'propensity', y_col, t_col}]

  x_0 = data_0[cols]
  x_1 = data_1[cols]

  for c in cols:
    x_0[c] = x_0[c].astype(int)
    x_1[c] = x_1[c].astype(int)

  scaler_0 = MinMaxScaler()
  scaler_0.fit(x_0)
  x_0_normalized = scaler_0.transform(x_0)

  scaler_1 = MinMaxScaler()
  scaler_1.fit(x_1)
  x_1_normalized = scaler_1.transform(x_1)

  y_1 = data_1[y_col].to_numpy()
  y_0 = data_0[y_col].to_numpy()

    
  nbrs_0 = NearestNeighbors(n_neighbors=num_neighboors, algorithm='ball_tree').fit(x_0_normalized)
  _, indices_0 = nbrs_0.kneighbors(x_1_normalized)

  nbrs_1 = NearestNeighbors(n_neighbors=num_neighboors, algorithm='ball_tree').fit(x_1_normalized)
  _, indices_1 = nbrs_1.kneighbors(x_0_normalized)

  ITE_0_list = []

  for i, indices_0 in enumerate(indices_0):
    closest_y_0_avg = np.mean(y_0[indices_0])
    ITE = y_1[i] - closest_y_0_avg
    ITE_0_list.append(ITE)

  ITE_1_list = []

  for i, indices_1 in enumerate(indices_1):
    closest_y_1_avg = np.mean(y_1[indices_1])
    ITE = closest_y_1_avg - y_0[i]
    ITE_1_list.append(ITE)
  
  ATE = np.mean(ITE_0_list + ITE_1_list)
  return ATE

In [10]:
def get_all_estimations(data, start_year, t_limit=6):
  transformed_data_control, transformed_data_treated = transform_data(data, start_year, t_limit)

  results = dict()
  results['IPW'] = dict()
  results['IPW']['ATT'] = calc_IPW_ATT(transformed_data_control, transformed_data_treated, start_year)
  results['IPW']['ATE'] = calc_IPW_ATE(transformed_data_control, transformed_data_treated, start_year)

  results['Slearner'] = dict()
  results['Slearner']['ATT'] = calc_ATT_Slearner(data, start_year)
  results['Slearner']['ATE'] = calc_ATE_Slearner(data, start_year)

  results['Tlearner'] = dict()
  results['Tlearner']['ATT'] = calc_ATT_Tlearner(data, start_year)
  results['Tlearner']['ATE'] = calc_ATE_Tlearner(data, start_year)

  results['matching'] = dict()
  num_neighboors = 2
  results['matching']['ATT'] = calc_ATT_matching(data, start_year, num_neighboors)
  results['matching']['ATE'] = calc_ATE_matching(data, start_year, num_neighboors)

  return results

In [ ]:
data = pd.read_csv('2009_2010_data.csv')
data = preprocess_data(data, 2009, 6)
results_2009_2010 = get_all_estimations(data, 2009)

In [19]:
results_2009_2010

{'IPW': {'ATT': -0.06092193721630501, 'ATE': -0.18022658743835943},
 'Slearner': {'ATT': -0.06831139976801309, 'ATE': -0.06623586994755692},
 'Tlearner': {'ATT': -0.06035876006124587, 'ATE': -0.009077854482146397},
 'matching': {'ATT': -0.06823144104803494, 'ATE': -0.0900486057815298}}

In [ ]:
data = pd.read_csv('2010_2011_data.csv')
data = preprocess_data(data, 2010, 6)
results_2010_2011 = get_all_estimations(data, 2010)

In [21]:
results_2010_2011

{'IPW': {'ATT': -0.11082910278531433, 'ATE': -0.2850867922245144},
 'Slearner': {'ATT': -0.11733213440011481, 'ATE': -0.1169025290716936},
 'Tlearner': {'ATT': -0.08843769123012075, 'ATE': -0.08943324414182813},
 'matching': {'ATT': -0.11882893226176808, 'ATE': -0.07103100980652001}}

In [22]:
def get_confidence_intervals(data, start_year, B=1000, t_limit=6):
  results = dict()
  results['IPW'] = dict()
  results['IPW']['ATT'] = []
  results['IPW']['ATE'] = []
  results['Slearner'] = dict()
  results['Slearner']['ATT'] = []
  results['Slearner']['ATE'] = []
  results['Tlearner'] = dict()
  results['Tlearner']['ATT'] = []
  results['Tlearner']['ATE'] = []
  results['matching'] = dict()
  results['matching']['ATT'] = []
  results['matching']['ATE'] = []

  for b in range(B):
    data_sample = data.sample(frac=1, replace=True)
    res_sample = get_all_estimations(data_sample, start_year)
    results['IPW']['ATT'].append(res_sample['IPW']['ATT'])
    results['IPW']['ATE'].append(res_sample['IPW']['ATE'])
    
    results['Slearner']['ATT'].append(res_sample['Slearner']['ATT'])
    results['Slearner']['ATE'].append(res_sample['Slearner']['ATE'])
    
    results['Tlearner']['ATT'].append(res_sample['Tlearner']['ATT'])
    results['Tlearner']['ATE'].append(res_sample['Tlearner']['ATE'])

    results['matching']['ATT'].append(res_sample['matching']['ATT'])
    results['matching']['ATE'].append(res_sample['matching']['ATE'])
  
  results['IPW']['ATT'] = (sorted(results['IPW']['ATT'])[24], sorted(results['IPW']['ATT'])[974])
  results['IPW']['ATE'] = (sorted(results['IPW']['ATE'])[24], sorted(results['IPW']['ATE'])[974])
  
  results['Slearner']['ATT'] = (sorted(results['Slearner']['ATT'])[24], sorted(results['Slearner']['ATT'])[974])
  results['Slearner']['ATE'] = (sorted(results['Slearner']['ATE'])[24], sorted(results['Slearner']['ATE'])[974])
  
  results['Tlearner']['ATT'] = (sorted(results['Tlearner']['ATT'])[24], sorted(results['Tlearner']['ATT'])[974])
  results['Tlearner']['ATE'] = (sorted(results['Tlearner']['ATE'])[24], sorted(results['Tlearner']['ATE'])[974])

  results['matching']['ATT'] = (sorted(results['matching']['ATT'])[24], sorted(results['matching']['ATT'])[974])
  results['matching']['ATE'] = (sorted(results['matching']['ATE'])[24], sorted(results['matching']['ATE'])[974])

  return results


In [ ]:
data = pd.read_csv('2009_2010_data.csv')
data = preprocess_data(data, 2009, 6)
results_ci_2009_2010 = get_confidence_intervals(data, 2009)

In [24]:
results_ci_2009_2010

{'IPW': {'ATT': (-0.1735009009446955, 0.04402623662138527),
  'ATE': (-0.4098587024412444, -0.03816965441676895)},
 'Slearner': {'ATT': (-0.18309828814338233, 0.03224340458416477),
  'ATE': (-0.18359275070350475, 0.03515625)},
 'Tlearner': {'ATT': (-0.17151825233702947, 0.04572824837530061),
  'ATE': (-0.22318333998498116, 0.18539760024446025)},
 'matching': {'ATT': (-0.230605738575983, 0.09770742358078603),
  'ATE': (-0.25326170376055257, 0.06280378613456127)}}

In [ ]:
data = pd.read_csv('2010_2011_data.csv')
data = preprocess_data(data, 2010, 6)
results_ci_2010_2011 = get_confidence_intervals(data, 2010)

In [26]:
results_ci_2010_2011

{'IPW': {'ATT': (-0.22122923039254871, -0.001689357403633629),
  'ATE': (-0.5349458759734506, -0.07524969152284748)},
 'Slearner': {'ATT': (-0.23375175561797754, -0.012759878615702479),
  'ATE': (-0.23437499999999997, -0.013301749271137026)},
 'Tlearner': {'ATT': (-0.19318979971096387, 0.012715347790432598),
  'ATE': (-0.23488595833645406, 0.24767883315661018)},
 'matching': {'ATT': (-0.29682179341657206, 0.034523809523809526),
  'ATE': (-0.2280678505168301, 0.09978796713490591)}}

In [27]:
def calc_before_and_after_sleep_difference_on_treated(path_to_data, start_year):
  data = pd.read_csv(path_to_data)
  data = data[(data['T_' + str(start_year)] < 6) & (data['T_' + str(start_year)] > 0)]
  data['T_' + str(start_year + 1)] = data['T_' + str(start_year + 1)].apply(lambda x: x == 6)
  data = data[(data['y_' + str(start_year)] > 0) & (data['y_' + str(start_year + 1)] > 0)]
  data_1 = data[data['T_' + str(start_year + 1)] == 1]
  return (data_1['y_' + str(start_year + 1)] - data_1['y_' + str(start_year)]).mean()

In [29]:
calc_before_and_after_sleep_difference_on_treated('2009_2010_data.csv', 2009)

-0.046153846153846156

In [31]:
calc_before_and_after_sleep_difference_on_treated('2010_2011_data.csv', 2010)

-0.07958477508650519